In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk as nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fileDownloaded = drive.CreateFile({'id':'1DStZxLyHzS3WaZHYi1c5zb1AaT4k2Tm7'})

In [ ]:
fileDownloaded.GetContentFile('news-data.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('news-data.csv')

df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
df.shape

(2225, 2)

## Null value treatment

In [ ]:
df.isna().sum()

category    0
text        0
dtype: int64

We found no null values

# Duplicate value treatment

In [ ]:
df.duplicated().sum()

99

In [ ]:
df = df.drop_duplicates()
df.shape

(2126, 2)

In [ ]:
df = df.reset_index()

In [ ]:
df['category'].value_counts()

sport            504
business         503
politics         403
entertainment    369
tech             347
Name: category, dtype: int64

Its seems like our dataset is a bit balanced

In [ ]:
print('No.of unique text rows: ',len(df['text'].unique()))
print()
print('Total No.of text rows: ', df.shape[0])

No.of unique text rows:  2126

Total No.of text rows:  2126


Here we found unique text in our text lables

In [ ]:
df['text'].apply(lambda x: len(x.split(' '))).sum()

893008

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['text'] = df['text'].apply(clean_text)

In [ ]:
df['text'] = df['text'].str.replace('\d+', '')

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 30081 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2126, 250)


In [ ]:
Y = pd.get_dummies(df['category']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2126, 5)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1913, 250) (1913, 5)
(213, 250) (213, 5)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 250, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 5)                 505       
Total params: 5,080,905
Trainable params: 5,080,905
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/10
27/27 [==============================] - 29s 895ms/step - loss: 1.5913 - accuracy: 0.2571 - val_loss: 1.3194 - val_accuracy: 0.4271
Epoch 2/10
27/27 [==============================] - 23s 851ms/step - loss: 1.2595 - accuracy: 0.5351 - val_loss: 1.0718 - val_accuracy: 0.4271
Epoch 3/10
27/27 [==============================] - 23s 864ms/step - loss: 0.8898 - accuracy: 0.5942 - val_loss: 1.0237 - val_accuracy: 0.4896
Epoch 4/10
27/27 [==============================] - 23s 858ms/step - loss: 0.7825 - accuracy: 0.7025 - val_loss: 0.8781 - val_accuracy: 0.6875
Epoch 5/10
27/27 [==============================] - 23s 858ms/step - loss: 0.5198 - accuracy: 0.8609 - val_loss: 0.4975 - val_accuracy: 0.7500
Epoch 6/10
27/27 [==============================] - 23s 862ms/step - loss: 0.3184 - accuracy: 0.8854 - val_loss: 0.5627 - val_accuracy: 0.7969
Epoch 7/10
27/27 [==============================] - 24s 871ms/step - loss: 0.2142 - accuracy: 0.9619 - val_loss: 0.2809 - val_accuracy: 0.9531

In [ ]:
pred = model.predict(X_test)

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7/7 [==============================] - 1s 58ms/step - loss: 0.1863 - accuracy: 0.9624
Test set
  Loss: 0.186
  Accuracy: 0.962
